In [1]:
!pip install transformers torchaudio jiwer

In [6]:
!pip install pyctcdecode kenlm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184413 sha256=68a46658618adf4f255d48f1ed34df63fe9a1528872e216831835d00533b5f8b
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm


In [2]:
import multiprocessing as mp

try:
   mp.set_start_method('spawn', force=True)
   print("spawned")
except RuntimeError:
   pass

spawned


In [3]:
import torch

torch.multiprocessing.set_start_method('spawn', force=True)

In [4]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import AutoModelForCTC, AutoProcessor
import torchaudio
import jiwer
import pandas as pd
import kenlm

# Load the pre-trained model and processor
#model_name = "facebook/wav2vec2-base-960h"
model_name = "patrickvonplaten/wav2vec2-base-960h-4-gram"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name).to("cuda")
#


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizati

In [5]:
# Set the LibriSpeech dataset configuration
librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

100%|██████████| 331M/331M [00:33<00:00, 10.4MB/s]


In [6]:
# Process each example in the dataset
def evaluation(librispeech_dataset):
  predictions = []
  ground_truths = []
  ids = []
  for i in range(len(librispeech_dataset)):
      waveform, sample_rate, utterance, speaker_id, sentence_id, number = librispeech_dataset[i]

      # Preprocess the audio file
      input_values = processor(waveform.numpy(), sampling_rate = sample_rate,
                               return_tensors="pt").input_values.to("cuda")

      # Perform inference
      with torch.no_grad():
          logits = model(input_values).logits

      # Decode the logits to obtain the transcriptions
      #predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(logits.cpu().numpy()).text[0]
      predictions.append(transcription)
      ground_truths.append(utterance)
      prefix = "0" * (4-len(str(number)))
      ids.append(f"{speaker_id}-{sentence_id}-{prefix}{number}")
  return predictions, ground_truths, ids

In [7]:
predictions, ground_truths, ids = evaluation(librispeech_dataset)

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently suppo

In [8]:
jiwer.wer(ground_truths, predictions)

0.02601947656725502

In [9]:
librispeech_other_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-other", download=True)

100%|██████████| 314M/314M [00:28<00:00, 11.3MB/s]


In [10]:
predictions_other, ground_truths_other, ids_other = evaluation(librispeech_other_dataset)

Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently supported in this platform. Falling back to sequential decoding.
Parallel batch decoding is not currently suppo

In [11]:
jiwer.wer(ground_truths_other, predictions_other)

0.064726897579428

In [12]:
wer_clean = []
for i,g, p in zip(ids, ground_truths, predictions):
  wer_clean.append((i,jiwer.wer(g, p),g,p))

In [13]:
wer_clean_results = sorted(wer_clean,key=lambda tup: tup[1], reverse=True)
d = pd.DataFrame(wer_clean_results, columns = ['id', 'wer', 'ground_truth', 'prediction'])
d.to_csv('wav2vec4gram_clean.csv', index=False)

In [14]:
wer_other = []
for i,g, p in zip(ids_other, ground_truths_other, predictions_other):
  wer_other.append((i,jiwer.wer(g, p),g,p))

In [15]:
wer_other_results = sorted(wer_other,key=lambda tup: tup[1], reverse=True)
d1 = pd.DataFrame(wer_other_results, columns = ['id', 'wer', 'ground_truth', 'prediction'])
d1.to_csv('wav2vec4gram_other.csv', index=False)